In [1]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath

import a_quantum_stateology_lib as qstates
import b_quantum_geometries_lib as gij
import c_matrix_analysis_lib as matAnsys 
import d_spin_representations_lib as su2
import e_maxent_solver_lib as me

from datetime import datetime

In [2]:
N=6; chain_type = "XY"; k_B = 1; temp = 1e+1; temp_ref = 1e+1

### One body local spin operators
spin_ops_list = su2.one_body_spin_ops(size = N)
identity_op = spin_ops_list[0][0]

### Heisenberg Hamiltonian
Hamiltonian_paras = [-1e-1, -2.5e-1, 0., .5e-1]
H_loc = su2.Heisenberg_Hamiltonian(op_list = spin_ops_list, chain_type = chain_type,
                                           size = N, Hamiltonian_paras = Hamiltonian_paras,
                                           closed_bcs = True, visualization = False)

### Generating classical observables

cl_ops, label_ops = su2.classical_ops(size = N, Hamiltonian = H_loc,
                                                op_list = spin_ops_list, centered_x_op = False)

## The Hamiltonian is added a perturbation term 
perturbation_strength = max(Hamiltonian_paras)/2

HH = H_loc + perturbation_strength * cl_ops["x_op"]

In [3]:
# Exact evolution
beta = 1/temp
xmean = 10; nmean = 10; weight_H = 1e2; weight_x = 1e-2; weight_n = 1e-2

K0 = weight_H * beta * HH + weight_x * (cl_ops["x_op"] - xmean)**2 + weight_n * (cl_ops["n_oc_op"] - nmean)**2
rho0 = matAnsys.safe_expm_and_normalize(K = K0, return_free_energy = False)

In [4]:
qutip.expect(rho0, cl_ops["n_oc_op"])

2.5221775081807545

In [5]:
subset_of_observables = {}; subset_label_ops = []
subset_of_observables["x_op_sq"] = cl_ops["p_op"]; subset_label_ops.append("x_op_sq")
subset_of_observables["n_oc_op"] = cl_ops["n_oc_op"]; subset_label_ops.append("n_oc_op")
subset_of_observables["n_oc_op_sq"] = cl_ops["n_oc_op"]**2; subset_label_ops.append("n_oc_op_sq")
subset_of_observables["p_op"] = cl_ops["p_op"]; subset_label_ops.append("p_op")

In [6]:
ts = np.linspace(1e-1, 4., 500)
exact_ev = qutip.mesolve(H = HH,
                         rho0 = rho0,
                         tlist = ts,
                         e_ops = subset_of_observables
                         )

In [11]:
def quantum_dynamics(Hamiltonian, K0, basis, timespan, depth, beta, HB_B0):
    print("*** 1. Starting at", datetime.now)
    results = {}
    print("*** 2. Processing initial state of the system")
    print("*** 3. Building Hierarchical Basis of depth l = ", depth)
    HBasis_local = matAnsys.build_Hierarchical_Basis(Hamiltonian = Hamiltonian,
                                                     seed_operator = HB_B0[-1], 
                                                     depth = depth,
                                                     tol = 1e-5, verbose = False)
    Hierarchical_B = []; del HB_B0[-1]
    Hierarchical_B = HB_B0  + HBasis_local
    print("*** 4. Processing Manifold Restricted Dynamics.\n     4A. Fetching categories of inner products.\n ") 
    print("        ###. Solving with Exact Geometry")
    res_exact, rhot = me.solve_exact(Hamiltonian = Hamiltonian,
                                     K0 = K0, 
                                     timespan = timespan,
                                     return_evolved_rhos = True,
                                     return_qutip_res_obj = True)
    K_at_timet = res_exact.states
    print("        ###. Projected Max-Ent solution with: ")
    print("                                            A. KMB-geometry")
    instantaneous_KMB_geometry = [gij.fetch_kubo_int_inner_prod(sigma = rho) for rho in rhot]
    print("                                            B. instantaneous correlation geometry")
    instantaneous_corr_geometry = [gij.fetch_corr_inner_prod(sigma = rho) for rho in rhot]
    
    print("     4B. Computing evolutions and Projections.\n ")
    print("        ###. 1. KMB Geometry")
    
    sp = [gij.fetch_kubo_int_inner_prod(sigma) for sigma in rhot]
    orth_basis = [matAnsys.orthogonalize_basis(Hierarchical_B, sp_at_timet) for sp_at_timet in sp]
    result["KMB"] = [matAnsys.instantaneous_proj(rho_at_timet, orth_basis_at_timet, kmb_sp, kmb_sp)
                     for rho_at_timet, orth_basis_at_timet, kmb_sp in zip(rhot, orth_basis_at_timet, instantaneous_KMB_geometry)]
    
    print("        ###. 2. Hilbert Schmidt Geometry")
    sp = gij.fetch_HS_scalar_product()
    orth_basis = gij.orthogonalize_basis(Hierarchical_B, sp)
    result["HS"] = [matAnsys.static_proj(rho_at_timet, orth_basis, sp, kmb_sp)
                    for rho_at_timet, kmb_sp in zip(rhot, instantaneous_KMB_geometry)
                        ]
    
    print("        ###. 3. Stationary Correlation Geometry")
    sp = gij.fetch_corr_inner_prod(state)
    orth_basis = gij.orthogonalize_basis(Hierarchical_B, sp)
    result["corr (stationary)"] = [matAnsys.static_proj(rho_at_timet, orth_basis, sp, kmb_sp)
                                   for rho_at_timet, kmb_sp in zip(rhot, instantaneous_KMB_geometry)]
    print("        ###. 4. Instantaneous Correlation Geometry")
    
    result["corr (instataneous)"] = [matAnsys.instaneous_proj(rho_at_timet, orth_basis, corr_sp, kmb_sp)
                                     for rho_at_timet, kmb_sp in zip(rhot, instantaneous_KMB_geometry)]
    
    print("***. Test 1: Testing the Hermiticity of the Basis")
    

In [13]:
ts = np.linspace(0, 2.0,100)
HB_B0 = [identity_op, H_loc, cl_ops["n_oc_op"], cl_ops["n_oc_op"]**2, cl_ops["x_op"]]
quantum_dynamics(Hamiltonian = HH, K0 = K0, basis = HB_B0, timespan  = ts, depth = 4, beta = beta, HB_B0 = HB_B0)

*** 1. Starting at <built-in method now of type object at 0x00007FFD2931B650>
*** 2. Processing initial state of the system
*** 3. Building Hierarchical Basis of depth l =  4
*** 4. Processing Manifold Restricted Dynamics.
     4A. Fetching categories of inner products.
 
        ###. Solving with Exact Geometry
        ###. Projected Max-Ent solution with: 
                                            A. KMB-geometry
                                            B. instantaneous correlation geometry
     4B. Computing evolutions and Projections.
 
        ###. 1. KMB Geometry


NameError: name 'innerprod' is not defined

In [14]:
evals, evecs = HB_B0[-1].eigenstates()

In [21]:
res_exact, rhot = me.solve_exact(Hamiltonian = HH,
                                     K0 = K0, 
                                     timespan = ts,
                                     return_evolved_rhos = True,
                                     return_qutip_res_obj = True)

In [29]:
rhot[2]

(Quantum object: dims = [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape = (64, 64), type = oper, isherm = True
 Qobj data =
 [[ 0.0010003 +0.00000000e+00j  0.        +0.00000000e+00j
    0.        +0.00000000e+00j ...  0.        +0.00000000e+00j
    0.        +0.00000000e+00j -0.00325972+1.97532398e-06j]
  [ 0.        +0.00000000e+00j  0.00241424+0.00000000e+00j
    0.00280426+3.28479660e-06j ...  0.00645906+2.29592971e-05j
    0.00781489+3.31444458e-05j  0.        +0.00000000e+00j]
  [ 0.        +0.00000000e+00j  0.00280426-3.28479660e-06j
    0.00326074+0.00000000e+00j ...  0.00749856+1.78707179e-05j
    0.00907371+2.78546191e-05j  0.        +0.00000000e+00j]
  ...
  [ 0.        +0.00000000e+00j  0.00645906-2.29592971e-05j
    0.00749856-1.78707179e-05j ...  0.01845093+0.00000000e+00j
    0.02234004+1.53384481e-05j  0.        +0.00000000e+00j]
  [ 0.        +0.00000000e+00j  0.00781489-3.31444458e-05j
    0.00907371-2.78546191e-05j ...  0.02234004-1.53384481e-05j
    0.02742032+0.000